In [1]:
from datasets import load_dataset

# full dataset
ds = load_dataset("bigcode/bigcodebench", split="v0.1.4")

In [2]:
ds

Dataset({
    features: ['task_id', 'complete_prompt', 'instruct_prompt', 'canonical_solution', 'code_prompt', 'test', 'entry_point', 'doc_struct', 'libs'],
    num_rows: 1140
})

In [7]:
ds.to_json('../datasets/bigcodebench/raw/data_BigCodeBench.jsonl')

Creating json from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

6953834

In [10]:
import os
print(os.environ.get('BIGCODEBENCH_OVERRIDE_PATH', None))

None


In [6]:
for col, dtype in ds.features.items():
    print(col, dtype)

task_id Value(dtype='string', id=None)
complete_prompt Value(dtype='string', id=None)
instruct_prompt Value(dtype='string', id=None)
canonical_solution Value(dtype='string', id=None)
code_prompt Value(dtype='string', id=None)
test Value(dtype='string', id=None)
entry_point Value(dtype='string', id=None)
doc_struct Value(dtype='string', id=None)
libs Value(dtype='string', id=None)


In [4]:
prompt=ds['complete_prompt'][0]
print(prompt)

import itertools
from random import shuffle

def task_func(numbers=list(range(1, 3))):
    """
    Calculates the average of the sums of absolute differences between each pair of consecutive numbers 
    for all permutations of a given list. Each permutation is shuffled before calculating the differences.

    Args:
    - numbers (list): A list of numbers. Default is numbers from 1 to 10.
    
    Returns:
    float: The average of the sums of absolute differences for each shuffled permutation of the list.

    Requirements:
    - itertools
    - random.shuffle

    Example:
    >>> result = task_func([1, 2, 3])
    >>> isinstance(result, float)
    True
    """



In [7]:
ds.save_to_disk('../datasets/bigcodebench/raw')

Saving the dataset (0/1 shards):   0%|          | 0/1140 [00:00<?, ? examples/s]

In [1]:
!pip install gradio_client

Defaulting to user installation because normal site-packages is not writeable
  Using cached gradio_client-1.10.3-py3-none-any.whl.metadata (7.1 kB)
  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)
  Using cached huggingface_hub-0.33.0-py3-none-any.whl.metadata (14 kB)
  Using cached websockets-15.0.1-cp313-cp313-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached hf_xet-1.1.5-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (879 bytes)
Using cached gradio_client-1.10.3-py3-none-any.whl (323 kB)
Using cached huggingface_hub-0.33.0-py3-none-any.whl (514 kB)
Using cached hf_xet-1.1.5-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Using cached filelock-3.18.0-py3-none-any.whl (16 kB)
   ━━━━━━━━━━━

In [2]:
from gradio_client import Client, handle_file

client = Client("https://43851cbb177855a4da.gradio.live/")
result = client.predict(
		split="complete",
		subset="full",
		samples=handle_file("/scratch3/workspace/wenlongzhao_umass_edu-reason/dev_kedar/Small-LLM-Reasoning/notebooks/output.jsonl"),
		pass_k="1,5,10",
		parallel=-1,
		min_time_limit=1,
		max_as_limit=30720,
		max_data_limit=30720,
		max_stack_limit=10,
		calibrated=True,
		check_gt_only=False,
		no_gt=False,
		selective_evaluate="",
		api_name="/predict"
)
print(result)

Loaded as API: https://43851cbb177855a4da.gradio.live/ ✔


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [3]:
print(type(result))

<class 'tuple'>


In [11]:
print(len(result))
print(result[1])

2
{'pass@1': 0.21666666666666667, 'model': 'output.jsonl', 'split': 'complete', 'subset': 'full', 'calibrated': True, 'gt_pass_rate': 0.9982456140350877, 'failed_tasks': ['BigCodeBench/205', 'BigCodeBench/363']}


In [27]:
result[0]['eval']['BigCodeBench/5']

[{'task_id': 'BigCodeBench/5',
  'solution': 'import random\nimport math\ndef task_func(LETTERS=[chr(i) for i in range(97, 123)]):\n\n    pass\nimport random\nimport math',
  'status': 'fail',
  'details': {'test_empty_letters': 'Traceback (most recent call last):\n  File "__test__.py", line 41, in test_empty_letters\nAssertionError: None != {}\n',
   'test_custom_letters': 'Traceback (most recent call last):\n  File "__test__.py", line 26, in test_custom_letters\nAttributeError: \'NoneType\' object has no attribute \'keys\'\n',
   'test_default_letters': 'Traceback (most recent call last):\n  File "__test__.py", line 19, in test_default_letters\nAttributeError: \'NoneType\' object has no attribute \'keys\'\n',
   'test_known_values': 'Traceback (most recent call last):\n  File "/work/pi_mccallum_umass_edu/kchimmad_umass_edu/conda_envs/bigcodebench/lib/python3.8/unittest/mock.py", line 1325, in patched\n    return func(*newargs, **newkeywargs)\n  File "__test__.py", line 52, in test_kn

In [20]:
print(result[0]['eval']['BigCodeBench/205'][0]['solution'])

import subprocess
from multiprocessing import Pool
def execute_command(command):
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
    output, _ = process.communicate()
    return output
def task_func(commands):

    pass
import subprocess
from multiprocessing import Pool

def execute_command(command):
    """
    Executes a shell command and returns its output.

    Parameters:
        command (str): A shell command to be executed.

    Returns:
        bytes: The output of the command as a byte string.

    Notes:
    - If the command fails, the function will return an empty byte string.
    """
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
    output, _ = process.communicate()
    return output

def task_func(commands):
    """
    Executes a list of shell commands in parallel using multiprocessing, and collects their outputs.

    Parameters:
        commands (list): A list of sh